BEST RESULTS SO FAR

In [1]:
import os
import sys

import pandas as pd

from datetime import datetime, timedelta
from pathlib import Path


# Add the project root to the Python path
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.append(project_root)
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import VecNormalize, DummyVecEnv
from trading.environments.forex_env2_flat import ForexTradingEnv
from stable_baselines3.common.callbacks import EvalCallback
from stable_baselines3.common.monitor import Monitor
from data_management.dataset_manager import DatasetManager



all_tickers = [
                             'GBP_CHF', 'GBP_JPY', 'EUR_CHF', 'EUR_JPY', 'USD_CHF',
 
        
        ]

sequence_length =[ 4,
                  ]

eur_usd = 'EUR_USD'

for sequence in sequence_length:
    train_set = f'/Volumes/ssd_fat2/ai6_trading_bot/datasets/5min/best_dataframes/{eur_usd}_5T_indics_1H_norm.parquet'
    df = pd.read_parquet(train_set)
    dataset_manager = DatasetManager()
    train_df, val_df, test_df = dataset_manager.split_dataset(df, train_ratio=0.7, val_ratio=0.15, test_ratio=0.15)

    saving_path = f'/Volumes/ssd_fat2/ai6_trading_bot/optimization_sequence_length/'
    os.makedirs(saving_path, exist_ok=True)

    def make_train_env():
        env = ForexTradingEnv(
            df=train_df,
            pair=eur_usd,
            sequence_length=sequence,

        )
        env = Monitor(env)
        env = DummyVecEnv([lambda: env])
        env = VecNormalize(env, norm_obs=True, norm_reward=True)
        return env

    def make_eval_env():
        env = ForexTradingEnv(

            df=val_df,
            pair=eur_usd,
            sequence_length=sequence,
            # resample_interval='1h'
        )
        env = Monitor(env)
        env = DummyVecEnv([lambda: env])
        env = VecNormalize(env, norm_obs=True, norm_reward=False)
        env.training = False
        return env

    train_env = make_train_env()
    eval_env = make_eval_env()
    eval_callback = EvalCallback(
        eval_env,
        best_model_save_path=saving_path,
        log_path=saving_path,
        eval_freq=50_000,  # Adjust as needed
        n_eval_episodes=5,
        deterministic=True,
        render=False
    )

    model = PPO(
        'MlpPolicy',
        train_env,
        verbose=0,
        tensorboard_log=f'{saving_path}sequence_{sequence}_tensorboard/',
    )

    model.learn(
        total_timesteps=2_000_000,  # Adjust as needed
        callback=eval_callback
    )

    model.save(f'{saving_path}{sequence}_best_model.zip')
    train_env.save(f'{saving_path}{sequence}_vec_normalize.pkl')

Logging configuration loaded from /Users/floriankockler/Code/GitHub.nosync/ai6-gcp-bot/forex_trading_system/config/logging_config.yaml
2024-12-04 12:33:40 - oandapyV20.oandapyV20 - INFO - oandapyV20.py:207 - setting up API-client for environment practice
Dataset split sizes:
Training: 1227053 samples (70.0%)
Validation: 262940 samples (15.0%)
Test: 262940 samples (15.0%)
2024-12-04 12:35:39 - ForexEnv2_flat - INFO - forex_env2_flat.py:320 - 
Episode Summary:
2024-12-04 12:35:39 - ForexEnv2_flat - INFO - forex_env2_flat.py:321 - Final Return: 19.09%
2024-12-04 12:35:39 - ForexEnv2_flat - INFO - forex_env2_flat.py:322 - Total PnL: 279683.44
2024-12-04 12:35:39 - ForexEnv2_flat - INFO - forex_env2_flat.py:323 - Total Trades: 2993
2024-12-04 12:35:39 - ForexEnv2_flat - INFO - forex_env2_flat.py:324 - Winning Trades: 2612
2024-12-04 12:35:39 - ForexEnv2_flat - INFO - forex_env2_flat.py:325 - Win Rate: 87.27%
2024-12-04 12:35:39 - ForexEnv2_flat - INFO - forex_env2_flat.py:326 - Initial Bala